In [1]:
import pandas as pd

# データの読み込み
# ジムの利用履歴データ
DATASET_PATH = 'C://Users/toru/Work/dataset/31-40/'
path1 = DATASET_PATH + "use_log.csv"
path2 = "customer_join.csv"

uselog = pd.read_csv(path1)
customer = pd.read_csv(path2)

In [3]:
# 顧客データから必要なカラムを抽出
customer_clustering = customer[["mean", "median", "max", "min", "membership_period" ]]
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


### K-means法を使ったクラスタリング

In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 標準化を実施
sc = StandardScaler()
customer_clustering_sc = sc.fit_transform(customer_clustering)

# KMeans法を使用して４つのクラスに分類
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(customer_clustering_sc)
# 「clusters列」クラスを各レコードに格納
customer_clustering["cluster"] = clusters.labels_

# ４つのクラスに分類されたことを確認
print(customer_clustering["cluster"].unique())
customer_clustering.head()

KMeans(n_clusters=4, random_state=0)
[2 3 0 1]


<ipython-input-7-901a4df18a53>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_clustering["cluster"] = clusters.labels_


,mean,median,max,min,membership_period,cluster
0,4.833333,5.0,8,2,47,2
1,5.083333,5.0,7,3,47,2
2,4.583333,5.0,6,3,47,2
3,4.833333,4.5,7,2,47,2
4,3.916667,4.0,6,1,47,2


In [9]:
customer_clustering.columns = ["月内平均値", "月内中央値", "月内最大値", "月内最小値", "会員期間", "cluster"]
# クラス毎に集計
customer_clustering.groupby("cluster").count()

,月内平均値,月内中央値,月内最大値,月内最小値,会員期間
cluster,,,,,
0,1332,1332,1332,1332,1332
1,840,840,840,840,840
2,1249,1249,1249,1249,1249
3,771,771,771,771,771


In [13]:
# クラス毎に平均値を出力
customer_clustering.groupby("cluster").mean()

,月内平均値,月内中央値,月内最大値,月内最小値,会員期間
cluster,,,,,
0,5.539535,5.391141,8.756006,2.702703,14.867868
1,8.061942,8.047024,10.014286,6.175000,7.019048
2,4.677561,4.670937,7.233787,2.153723,36.915933
3,3.065504,2.900130,4.783398,1.649805,9.276265


#### 上記の出力からわかること
- 1は会員期間が短く、利用頻度が高い
- 3は会員期間が短く、利用頻度が低い